In [1]:
# This notebook shows how to compute the expectation value of a one and two qubit hamiltonian.

In [2]:
import numpy as np
import scipy.linalg

In [3]:
# Some helper functions we will use throughout the notebook

def dagger(state):
    return np.transpose(np.conj(state))

In [4]:
# Define the Pauli Matrices

pauli_I = np.array([[1,0],[0,1]])
pauli_X = np.array([[0,1],[1,0]])
pauli_Y = np.array([[0,-1j],[1j,0]])
pauli_Z = np.array([[1,0],[0,-1]])

pauli_matrices = [pauli_I, pauli_X, pauli_Y, pauli_Z]

# Define Hadamard Gate
hadamard = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])

# Define Phase gate
s_phase  = np.array([[1,0],[0,1j]])

In [5]:
# Example Hamiltonians for Non-Interacting Case
ham_0 = np.array([[1,0],[0,2]])
ham_1 = np.array([[0,2*1j],[-2*1j, 0]])

# The total hamiltonian is found as follows:
ham_tot = np.kron(ham_0, np.identity(2)) + np.kron(np.identity(2), ham_1)

# We know each of the hamiltonians can be written as a sum of pauli matrices with real coefficients

# Here's a function for decomposing single qubit hamiltonians with hilbert space dimension 2
# This is based on the method given on page 4 of the lecture 4 handout
def find_pauli_coeffs(ham):
    pauli_coeffs = np.array([np.trace(ham@pauli_I), np.trace(ham@pauli_X), np.trace(ham@pauli_Y), np.trace(ham@pauli_Z)])
    return np.real((1/2)*pauli_coeffs)

# Consider two examples, you can confirm these by hand
ham_0_coeffs  = find_pauli_coeffs(ham_0)
print(f"ham_0 = {ham_0_coeffs[0]}*sigma_I + {ham_0_coeffs[1]}*sigma_X + {ham_0_coeffs[2]}*sigma_Y + {ham_0_coeffs[3]}*sigma_Z")
ham_1_coeffs  = find_pauli_coeffs(ham_1)
print(f"ham_1 = {ham_1_coeffs[0]}*sigma_I + {ham_1_coeffs[1]}*sigma_X + {ham_1_coeffs[2]}*sigma_Y + {ham_1_coeffs[3]}*sigma_Z")

ham_0 = 1.5*sigma_I + 0.0*sigma_X + 0.0*sigma_Y + -0.5*sigma_Z
ham_1 = 0.0*sigma_I + 0.0*sigma_X + -2.0*sigma_Y + 0.0*sigma_Z


In [6]:
# The Variational Quantum Eigensolver algorithm aims to find the minimum eigenvalue or ground state energy of a hamiltonian 
# We can diagonalise our hamiltonian for a single qubit very easily and find the minimum eigenvalue
# VQE uses classical feedback based on the current measured expectation value to vary the state so that it approaches the ground state

# These two small examples below just use the numerically obtained ground state vectors to compute the expected values of the hamiltonian

# Ham 0 is already diagonal so these steps aren't stricly necessary
eigvals_ham_0, eigvecs_ham_0 = scipy.linalg.eigh(ham_0)
print("HAM 0 EIGENVALUES AND EIGENVECTORS")
print(f"Ham 0 EigenValues: {eigvals_ham_0}")
print(f"Ham 0 EigenVectors: {eigvecs_ham_0}")
print(f"Sum of Pauli Expectations with Ground State Vector: {ham_0_coeffs[0]*dagger(eigvecs_ham_0[0,:])@pauli_I@eigvecs_ham_0[0,:] + ham_0_coeffs[3]*dagger(eigvecs_ham_0[0,:])@pauli_Z@eigvecs_ham_0[0,:]}")

# Ham 1 is not diagonal
# We need basis transformation to find the expected value of this hamiltonian
eigvals_ham_1, eigvecs_ham_1 = scipy.linalg.eigh(ham_1)
print("HAM 1 EIGENVALUES AND EIGENVECTORS")
print(f"Ham 1 EigenValues: {eigvals_ham_1}")
print(f"Ham 1 EigenVectors: {eigvecs_ham_1}")
print(f"Sum of Pauli Expectations with Ground State Vector: {ham_1_coeffs[2]*dagger(eigvecs_ham_1[0,:])@(hadamard@pauli_Z@hadamard)@eigvecs_ham_1[0,:]}")

HAM 0 EIGENVALUES AND EIGENVECTORS
Ham 0 EigenValues: [1. 2.]
Ham 0 EigenVectors: [[-1.  0.]
 [ 0.  1.]]
Sum of Pauli Expectations with Ground State Vector: 1.0
HAM 1 EIGENVALUES AND EIGENVECTORS
Ham 1 EigenValues: [-2.  2.]
Ham 1 EigenVectors: [[-0.70710678+0.j          0.70710678+0.j        ]
 [ 0.        -0.70710678j  0.        -0.70710678j]]
Sum of Pauli Expectations with Ground State Vector: (1.9999999999999993+0j)


In [7]:
# If we want to do the equivalent for a 2 qubit state and find its pauli string decomposition we can expand on what we wrote earlier

def find_pauli_coeffs_2qubit(ham):
    ham_pauli_coeffs = {}
    for indexA, current_pauli_A in enumerate(pauli_matrices):
        for indexB, current_pauli_B in enumerate(pauli_matrices):
            current_pauli_string   = np.kron(current_pauli_A, current_pauli_B)
            current_ham_projection = np.real((1/2**2)*np.trace(ham@current_pauli_string))
            if current_ham_projection != 0.0: #Let's only include non-zero coefficients in our expansion
                ham_pauli_coeffs.update({f"{indexA}{indexB}": current_ham_projection})
    return ham_pauli_coeffs

# Let's do an example with the total hamiltonian from the two non-interacting single qubit hamiltonians
# The output is given as a dictionary for legibility but isn't stricly necessary
# e.g. key 02 corresponds to sigma_I on system A and sigma_Y on system B and the value is the coefficient
# You should confirm this by hand
ham_tot_pauli_decomp = find_pauli_coeffs_2qubit(ham_tot)
print(ham_tot_pauli_decomp)

{'00': 1.5, '02': -2.0, '30': -0.5}


In [8]:
# To compute the expectaion value in the multi qubit case we need to apply the appropriate transformation for each pauli matrix
# We can only perform Z basis measurements so we must transform our qubits prior to measurement
# sigma_I = sigma_I
# sigma_X = H*sigma_Z*H
# sigma_Y = S*H*sigma_Z*H*S^{\dagger}
# sigma_Z = sigma_Z

# Let's write a function that returns the appropriate unitary transform given a key such as we generated above
def basis_transform_2qubit(pauli_string):
    system_A_pauli = int(pauli_string[0])
    system_B_pauli = int(pauli_string[1])
    
    system_A_transform = None
    system_B_transform = None
    
    if system_A_pauli  == 0:
        system_A_transform = pauli_I
    elif system_A_pauli == 1:
        system_A_transform = hadamard
    elif system_A_pauli == 2:
        system_A_transform = hadamard@dagger(s_phase)
    elif system_A_pauli == 3:
        system_A_transform = pauli_I
    
    if system_B_pauli == 0:
        system_B_transform = pauli_I
    elif system_B_pauli == 1:
        system_B_transform = hadamard
    elif system_B_pauli == 2:
        system_B_transform = hadamard@dagger(s_phase)
    elif system_B_pauli == 3:
        system_B_transform = pauli_I
        
    return np.kron(system_A_transform, system_B_transform)


basis_transform_2qubit('00')

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [9]:
# take in dictionary of key value pairs, take in a state vector
# for each key find the transform
# then compute  value * dagger(state)@dagger(transform)@sigma_Z@transform@state

def expectation_from_decomp(ham_pauli_decomp, state):
    
    total_expectation = 0.0
    for current_key in ham_pauli_decomp.keys():
        current_transform  = basis_transform_2qubit(current_key)
        transformed_state  = current_transform@state
        system_A_pauli = int(current_key[0])
        system_B_pauli = int(current_key[1])
        
        if system_A_pauli == 0:
            system_A_op = pauli_I
        else:
            system_A_op = pauli_Z
            
        if system_B_pauli == 0:
            system_B_op = pauli_I
        else:
            system_B_op = pauli_Z
            
        total_expectation += ham_pauli_decomp[current_key]*dagger(transformed_state)@np.kron(system_A_op, system_B_op)@transformed_state

    return total_expectation

eigvals_tot, eigvecs_tot = scipy.linalg.eigh(ham_tot)
print("HAM TOT EIGENVALUES AND EIGENVECTORS")
print(f"Ham tot EigenValues: {eigvals_tot}")
print(f"Ham tot EigenVectors: {eigvecs_tot}")

ground_state_tot = eigvecs_tot[:,0]
expectation_from_decomp(ham_tot_pauli_decomp, ground_state_tot)

HAM TOT EIGENVALUES AND EIGENVECTORS
Ham tot EigenValues: [-1.00000000e+00  2.22044605e-15  3.00000000e+00  4.00000000e+00]
Ham tot EigenVectors: [[ 0.70710678+0.j          0.        +0.j         -0.70710678-0.j
   0.        +0.j        ]
 [ 0.        +0.70710678j  0.        +0.j          0.        +0.70710678j
   0.        +0.j        ]
 [ 0.        +0.j          0.70710678+0.j          0.        +0.j
  -0.70710678-0.j        ]
 [ 0.        +0.j          0.        +0.70710678j  0.        +0.j
   0.        +0.70710678j]]


(-1+0j)